<a href="https://colab.research.google.com/github/LongxiZhou/DLPE-method/blob/master/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before we start, please access the [shared data](https://drive.google.com/drive/folders/16ZvZfhqMmuF7wqNPKUOntw2P-Mfx5C4l?usp=sharing) and do following steps.

1. Go to Shared with me in Google Drive.
2. Select the folder or file you want to acess.
3. Right click on it and choose Add shortcut to drive.
4. A pop-up window will apear, Select MyDrive then click on Add Shortcut.

Now we load the shared data in the colab.

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/DLPE method"

Mounted at /content/drive
 DLPE-methods   example_output	   'Reproduce Follow-up Dataset'
 example_data   outside_share.zip   trained_models


We will git clone the source data to colab now. Run the following code will replace the "trained_models/" and "example_data/" with shared data in Google Drive.

In [ ]:
!git clone https://github.com/LongxiZhou/DLPE-method
!rm -rf "/content/DLPE-method/trained_models"
!rm -rf "/content/DLPE-method/example_data"
!ln -s "/content/drive/My Drive/DLPE method/trained_models/" "/content/DLPE-method/"
!ln -s "/content/drive/My Drive/DLPE method/example_data/" "/content/DLPE-method/"

Cloning into 'DLPE-method'...
remote: Enumerating objects: 969, done.
remote: Counting objects: 100% (361/361), done.
remote: Compressing objects: 100% (205/205), done.
remote: Total 969 (delta 171), reused 338 (delta 149), pack-reused 608
Receiving objects: 100% (969/969), 21.12 MiB | 10.17 MiB/s, done.
Resolving deltas: 100% (397/397), done.


Build the conda enviroment with the following command.

In [ ]:
!pip install condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:58
🔁 Restarting kernel...


Check conda!

In [ ]:
!conda --version
!which conda

conda 4.9.2
/usr/local/bin/conda


Build the DLPE environment, this step takes long

In [ ]:
!conda env update -n base -f "/content/DLPE-method/environment.yml"

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - WARNING conda.core.solve:_add_specs(611): pinned spec cudatoolkit=11.1 conflicts with explici

In [ ]:
!pip install ipykernel

We are happy now!

Open **/content/DLPE-method/dcm_to_enhanced.py**, follow the instructions to change global parameters "trained_model_top_dict", "dcm_directory" and "enhance_array_output_directory". 

Following is an example of the file **/content/DLPE-method/dcm_to_enhanced.py**:

In [ ]:
import post_processing.parenchyma_enhancement as enhancement
import prediction.predict_rescaled as predictor
import Format_convert.dcm_np_converter as normalize
import Interface.visualize_example as visualize
import Tool_Functions.Functions as Functions

"""
each chest CT needs about one minute to be enhanced on one V100 GPU.
"""

batch_size = 4  # Need about 3 GB GPU Ram

trained_model_top_dict = '/content/DLPE-method/trained_models/'
# the directory where the trained models are saved

dcm_directory = '/content/DLPE-method/example_data/COVID-19 inpatient/'
# the directory where the dicom or dcm files for ONE chest CT scan

enhance_array_output_directory = '/content/DLPE-method/example_output/'
# the directory to save the enhanced ct data

predictor.top_directory_check_point = trained_model_top_dict

rescaled_ct_array = normalize.dcm_to_spatial_signal_rescaled(dcm_directory, wc_ww=(-600, 1600))
# you can set wc_ww to None if dcm or dicom contains information about correct lung window.

lung_mask = predictor.predict_lung_masks_rescaled_array(rescaled_ct_array, refine=False, batch_size=batch_size)

airway_mask = predictor.get_prediction_airway(rescaled_ct_array, lung_mask=lung_mask, semantic_ratio=0.02,
                                              refine_airway=False, batch_size=batch_size)
# refine_airway will leave one connected component (discard around 1% predicted positives) and cost about 30 sec.

blood_vessel_mask = predictor.get_prediction_blood_vessel(rescaled_ct_array, lung_mask=lung_mask, semantic_ratio=0.1,
                                                          refine_blood_vessel=False, batch_size=batch_size)
# refine_blood_vessel will leave one connected component (discard around 2% predicted positives) and cost about 60 sec.

DLPE_enhanced, w_l, w_w = enhancement.remove_airway_and_blood_vessel_general_sampling(rescaled_ct_array, lung_mask,
                                                                                      airway_mask, blood_vessel_mask, window=True)
w_l = round(w_l)  # the optimal window level for observing sub-visual parenchyma lesions
w_w = round(w_w)  # the window width for observing sub-visual parenchyma lesions
print("\n\n#############################################")
print("the scan level optimal window level is:", w_l, "(HU)")
print("recommend window width is:", w_w, "(HU)")
print("#############################################\n\n")

example_slice = visualize.generate_slice(rescaled_ct_array, DLPE_enhanced, airway_mask, blood_vessel_mask,
                                         slice_z=250, show=True)

Functions.image_save(example_slice, enhance_array_output_directory + 'slice image name.png', high_resolution=True)

Functions.save_np_array(enhance_array_output_directory, 'enhanced_ct_name', DLPE_enhanced, compress=True)

#######################################################################
# Here is the segmentation of sub-visual lesions for COVID-19 survivors.
# Note the segmentation model only for COVID-19 lesions.
#######################################################################

import prediction.predict_enhanced as predict_enhanced
predict_enhanced.top_directory_check_point = trained_model_top_dict

# mask for visible and sub-visual COVID-19 lesions for FOLLOW UP patients
follow_up_lesions = predict_enhanced.get_invisible_covid_19_lesion_from_enhanced(
    DLPE_enhanced, follow_up=True, batch_size=batch_size)

# mask for visible and sub-visual COVID-19 lesions for FOLLOW UP patients
inpatient_lesions = predict_enhanced.get_invisible_covid_19_lesion_from_enhanced(DLPE_enhanced, follow_up=False,
                                                                                 batch_size=batch_size)

# mask for visible COVID-19 lesions described in our TMI paper DOI: 10.1109/TMI.2020.3001810
visible_lesions = predictor.predict_covid_19_infection_rescaled_array(rescaled_ct_array, lung_mask=lung_mask,
                                                                      batch_size=batch_size)


CheckCheck GPUs

In [ ]:
!nvidia-smi

Sun May 22 16:48:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Then we can run it with the following command. 

**Please notice this step requires more than 24GB RAM, so a free colab account can not hold this.**

In [ ]:
%%bash
pwd
python DLPE-method/dcm_to_enhanced.py